
# Basic librat / RATstart operation


Librat is the library of function calls around which you can write your own code to do things such as read in and parse an object file, read in and parse camera, illumination files, waveband files and so on. However, RATstart (forerly start) is a wrapper code around these commands which gives you access to all the basic operations, and so is the de facto tool for doing simulations. The key things required to carry out a simulation are:

- A camera file
- An illumination file
- A waveband file
- An object file - this is always assumed to be the last file on the RATstart command line

Anything specific you want to do in any of these parts of the process is specified in these files. There are a limited number of additional command line options which either allow you to override a few key things in these files (the waveband file for example), or more usually are external to these things. Each of these can be passed through via the `-RAT` keyword. Examples are the ray tree depth (`-RATm`) , verbose level (`-RATv`), waveband file (`-RATsensor_wavebands`) etc.

---------------------------------



## Object example 1: planes and ellipsoids




Now, a simple scene object [`test/test_examples/first.obj`](test/test_examples/first.obj)

In [1]:
%%bash

cat <<EOF > test/test_examples/first.obj
# My first object file
mtllib plants.matlib 
usemtl white 
v 0 0 0 
v 0 0 1 
plane -1 -2 
!{
usemtl white 
!{ 
v 0 0 1000 
ell -1 30000 30000 1000 
!} 
!}
EOF

This object uses a material library [`plants.matlib`](test/test_examples/plants.matlib) that specifies the reflectance and transmittance properties of the scene materials.

In [2]:
%%bash

cat <<EOF > test/test_examples/plants.matlib
srm white refl/white.dat
EOF

In this example, the file contains the single line:

    srm white refl/white.dat

so there is only a single material of type `srm` (standard reflectance material - Lambertian reflectance (and/or transmittance). The material name is `white` and the (ASCII) file giving the spectral reflectance function is [`refl/white.dat`](test/test_examples/refl/white.dat). 

In [3]:
%%bash

mkdir -p test/test_examples/refl

cat <<EOF > test/test_examples/refl/white.dat
0 1 
10000 1
EOF

The file [`refl/white.dat`](test/test_examples/refl/white.dat) contains 2 columns: column 1 is 'wavelength' (really, a pseudo-wavelength in this case), column 2 is reflectance for that wavelength (wavelength units are arbitrary, but we usually use nm). 

In this case, the file specifies:

    0 1 
    10000 1

which is a reflectance of 1.0 for any wavelength (less than or equal to an arbitrary upper limit 10000). If the file specifies transmittance as well, this is given as a third column.

Looking back to [`test/test_examples/first.obj`](test/test_examples/first.obj), the line:

    mtllib plants.matlib 
    
tells the librat reader to load the 'material library' called [`plants.matlib`](test/test_examples/plants.matlib). First, it will look in the current directory for the file. If it doesn't find it there, it will see if the environment variable `MATLIB` is set. If so, it will look there next. 



## Environment variables

The following environmental variables can be used:


| Name | File types |
|:-:|:-:|
| `MATLIB` | material library e.g. `plants.matlib`, all materials defined in a material library e.g. `white.dat`|
| `ARARAT_OBJECT` | (extended) wavefront object files e.g. `first.obj`  |
| `DIRECT_ILLUMINATION` | spectral files for direct illumination: those defined in `-RATdirect` command line option |
| `RSRLIB` | sensor waveband files: those defined in `-RATsensor_wavebands` command line option |
| `BPMS_FILES` | Not used |
| `SKY_ILLUMINATION` | location of sky map image files: defined in `-RATskymap`  command line option |




You can set all of these to the same value, in which case the database of files is all defined relative to that point. This is the most typical use of `librat`. We illustrate this setup below for the `librat` distribution, where a set of examples use files from the directory `test/test_example`.

Additionally, the following environment variables can be set to extend the size of some aspects of the model. You would only need to use these in some extreme case.

| Name | Purpose |
|:-:|:-:|
| `MAX_GROUPS` | Maximum number of groups allowed (100000) |
| `PRAT_MAX_MATERIALS` | Maximum number of materials allowed (DEFAULT_PRAT_MAX_MATERIALS=1024 in `mtllib.h`) |
| `MAX_SUNS` | Maximum number of suns (180 in `rat.h`) |
| `MAX_OBJECT_TYPES` | Maximum number of types of object used (`sph`, `f`, etc): hardwired in `intersect_objects.h` at 16. Only used in `RATstart` option 8 |


In this case, we would want to set `MATLIB` to `test/test_examples` before invoking `librat`. In `bash` for example, this is done with:



In [4]:
%%bash

export MATLIB=test/test_examples

Let's put all of these into a shell called [`init.sh`](test/test_examples/init.sh):

In [5]:
%%bash

# create the init.sh file we want
outfile=test/test_examples/init.sh

cat <<EOF > $outfile
#!/bin/bash
#
# preamble 
#
ARCH=\$(uname -m)
BPMS=\${BPMS-\$(pwd)}
# set shell variables lib, bin, verbose
# with defaults in case not set 
lib=\${lib-"\$BPMS/lib/\$ARCH"}
bin=\${bin-"\$BPMS/bin/\$ARCH"}
VERBOSE=\${VERBOSE-1}

# set up required environment variables for bash
export LD_LIBRARY_PATH="\${lib}:\${LD_LIBRARY_PATH}"
export DYLD_LIBRARY_PATH="\${lib}:\${DYLD_LIBRARY_PATH}"
export PATH="\${bin}:\${PATH}"

# set up required environment variables for librat
export TEST=\${BPMS}/test/test_example
export MATLIB=\$TEST
export RSRLIB=\$TEST
export ARARAT_OBJECT=\$TEST
export DIRECT_ILLUMINATION=\$TEST
export BPMS_FILES=\$TEST


var=\$(diff "\$(which RATstart)" "\${bin}/RATstart")
#store diff exit code
exit_code=\$?
# remember that this is not the same as count of differences
lines_output_by_diff=\$(wc -l <<< "$var")

echo "\$var"

if ((\$exit_code == 0)); then
  if [ "\$VERBOSE" == "1" ] ; then 
    echo "RATstart found ok"
  fi
else
  cd $BPMS; make clean all test install 
fi
EOF

# set executable mode
chmod +x $outfile
# test run
$outfile


RATstart found ok


The object code line:

    usemtl white
    
tells `librat` to load the material named `white`. Since we defined that in [`plants.matlib`](test/test_examples/plants.matlib) as type `srm` with spectral file [`refl/white.dat`](test/test_examples/refl/white.dat), the material will have a Lambertian reflectance of `1.0` for all (up to 10000 units) wavelengths.

In [6]:
%%bash

cat <<EOF > test/test_examples/white.dat
1 1.0
1000 1.0
EOF

    mtllib plants.matlib 
    usemtl white 
    v 0 0 0 
    v 0 0 1 
    plane -1 -2 
    !{
    usemtl white 
    !{ 
    v 0 0 1000 
    ell -1 30000 30000 1000 
    !} 
    !}


The fields starting `v` in [`test/test_examples/first.obj`](test/test_examples/first.obj) denote a vertex (vector) (as in the standard wavefront format). This requires 3 numbers to be given after the `v` giving the {x,y,z} coordinates of the vector. Note that `v` fields can specify a *location* or *direction* vector. 

The fields `plane` and `ell` specify scene objects. We will look at a fuller range of such objects later, but these two allow for a simple scene specification. plane is an infinite planar object. It is defined by an intersection point (location vector) `I` and a direction vector `N`. These vectors need to be defined before a call is made to the object, so in this case, we define `I` as `0 0 0` and `N` as `0 0 1`, i.e. an x-y plane at z=0.

Thus `plane -1 -2` means 'define a plane with `N` given by the previous (`-1`) specified vector that goes through `I` given by the second to last specified vector (`-2`).'

`ell` is an ellipsoid object. Its description requires definition of: 

- the base (N.B. not the centre) of the ellipsoid (`-1` here, meaning the previously-defined vector - `0 0 1000` in this case); 
-  the semi-axis lengths in `x,y,z` directions (`30000 30000 1000` here).

so:

    v 0 0 1000 
    ell -1 30000 30000 1000

is in fact a spheroid of x-y semi-axis length 30000 units (arbitrary linear units) and z-semi-axis length 1000 units: a *prolate* spheroid that extends from `-30000` to `30000` in the x- and y-directions and from `1000` to `3000` in the z-direction. Not that the physical unit for these dimensions is arbitrary, but must be consistent throughout.


The fields `!{` and `!}` in [`test/test_examples/first.obj`](test/test_examples/first.obj) specify that a bounding box should be placed around objects contained within the brackets. This allows for efficient intersection tests in the ray tracing.

We now want to use the code `RATstart` to run `librat` functionality.

If you have compiled the code, the executable and library should be in the directory [`src`](src) as

    src/RATstart
    src/libratlib.[dll,so]
    
The suffix for the library will be `dll` on windows, and `so` on other operating systems. Lets just check they are there:

In [7]:
%%bash

ARCH=$(uname -m)
lib=${lib-"$BPMS/lib/$ARCH"}
bin=${bin-"$BPMS/bin/$ARCH"}

ls -l ${bin}/RATstart ${lib}/libratlib.*

-rwxr-xr-x  1 plewis  staff   49704 21 Oct 16:17 /Users/plewis/Documents/GitHub/librat/bin/x86_64/RATstart
-rwxr-xr-x  1 plewis  staff  437064 21 Oct 16:17 /Users/plewis/Documents/GitHub/librat/lib/x86_64/libratlib.so


Don't worry too much if its not there as we can make it when we need it.

In [8]:
%%bash

#
# shell preamble 
#

# set shell variables lib, bin, verbose
# with defaults in case not set 
ARCH=$(uname -m)
lib=${lib-"$BPMS/lib/$ARCH"}
bin=${bin-"$BPMS/bin/$ARCH"}
verbose=${verbose-1}

# set up required environment variables for bash
export LD_LIBRARY_PATH="${lib}:${LD_LIBRARY_PATH}"
export DYLD_LIBRARY_PATH="${lib}:${DYLD_LIBRARY_PATH}"
export PATH="${bin}:${PATH}"

# set up required environment variables for librat
export TEST=${BPMS}/test/test_example
export MATLIB=$TEST
export RSRLIB=$TEST
export ARARAT_OBJECT=$TEST
export DIRECT_ILLUMINATION=$TEST
export BPMS_FILES=$TEST

var=$(diff "$(which RATstart)" "${bin}/RATstart")
#store diff exit code
exit_code=$?
# remember that this is not the same as count of differences
lines_output_by_diff=$(wc -l <<< "$var")

echo "$var"

if (($exit_code == 0)); then
  if [ "$VERBOSE" == "1" ] ; then 
    echo "RATstart found ok"
  fi
else
  cd $BPMS; make clean all test install 
fi

## Object example 2: clones

---------------------------------



In [9]:
%%bash

cat <<EOF > test/test_examples/second.obj
!{
mtllib plants.matlib
v 0.000000 0.000000 0.000000
v 0.000000 0.000000 1.000000
usemtl full
plane -1 -2
!{
#define
g object 0
usemtl half
v 0 0 0
v 0 0 1
cyl -1 -2 0.1
sph -1 0.2
v -1 0 1
cyl -1 -2 0.1
!}
!{
clone 0 0 0 0 object 0
clone 0 1 0 90 object 0
clone -1 0 0 -90 object 0
!}
!}
EOF